In [1]:
import os
import openpyxl
import pandas as pd
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from datetime import datetime
from ipywidgets import interact
from openpyxl import Workbook

In [2]:
diretorio_lote = r"C:\Users\helym\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\00-INTELBRAS\000_Relatórios de Campo\001 RELATORIOS ENVIADOS\AC\1.0-CORRETIVA\2ªTRANCHE\LOTE_03"
path_excel_mov = r"C:\Users\helym\projetos_python\jupyter\OEM\content\BASE_MOVIMENTO_MATERIAIS_ONE_AC.xlsx"
path_excel_comp =r'C:\Users\helym\projetos_python\jupyter\OEM\content\lista_componente.xlsx'

In [3]:
def encontrar_duplicados(numocorrencia):
    duplicados = numocorrencia[numocorrencia.duplicated(subset='NUMOCORRENCIA')]
    return duplicados[['NUMOCORRENCIA']]

arquivos_pdf = []
for raiz, diretorios, arquivos in os.walk(diretorio_lote):
    for arquivo in arquivos:
        if arquivo.endswith('.pdf'):
            arquivos_pdf.append(os.path.join(raiz, arquivo))

string = pd.DataFrame(pd.Series(arquivos_pdf).str.extract(r"(\d+)_(\d+)_(.+)_(.+)_(.+)_(.+)"))
string.rename(columns={1: 'NUMOCORRENCIA'}, inplace=True)
numocorrencia= string[['NUMOCORRENCIA']].astype('int')

In [4]:
df_filtered_global_oem = None

def ler_planilha():
    """Lê uma planilha do Excel."""
    excluir_status = ['TREINAMENTO','DUPLICADO','CANCELADO ']
    df_bd = pd.read_excel(r"C:\Users\helym\projetos_python\jupyter\OEM\content\base_sip_Concluido.xlsx")
    df = df_bd[~df_bd['STATUS'].isin(excluir_status)]
    return df

def aplicar_filtros(rota_value, status_value, tipo_value, executor_value, origem_value, tipocausa_value, data_inicial, data_final):
    """Aplica filtros a uma planilha."""
    global df_filtered_global_oem
    df_filtered_global_oem = df[
        df["ROTA"].isin(rota_value) &
        df["STATUS"].isin(status_value) &
        df["TIPO"].isin(tipo_value) &
        df["EXECUTOR"].isin(executor_value) &
        df["ORIGEM"].isin(origem_value) &
        df["TIPOCAUSA"].isin(tipocausa_value) &
        (df["DATACONCLUSAO"] >= data_inicial) &
        (df["DATACONCLUSAO"] <= data_final)
    ]
    return df_filtered_global_oem

if __name__ == "__main__":
    df = ler_planilha()

    df["DATACONCLUSAO"] = pd.to_datetime(df["DATACONCLUSAO"]).dt.date

    rota_options = sorted(list(df['ROTA'].unique()))
    rota_filter = widgets.SelectMultiple(options=rota_options, description="Rotas:")

    status_options = sorted(list(df['STATUS'].unique()))
    status_filter = widgets.SelectMultiple(options=status_options, description="Status:")

    tipo_options = sorted(list(df['TIPO'].unique()))
    tipo_filter = widgets.SelectMultiple(options=tipo_options, description="Tipo:")

    executor_options = sorted(list(df['EXECUTOR'].unique()))
    executor_filter = widgets.SelectMultiple(options=executor_options, description="Usuário:")

    origem_options = sorted(list(df['ORIGEM'].unique()))
    origem_filter = widgets.SelectMultiple(options=origem_options, description="Origem:")

    tipocausa_options = sorted(list(df['TIPOCAUSA'].unique()))
    tipocausa_filter = widgets.SelectMultiple(options=tipocausa_options,descrition="Tipo Causa")

    data_inicial_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].min())
    data_final_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].max())

    aplicar_filter_button = widgets.Button(description="Aplicar filtros")

    def on_button_click(event):
        global df_filtered_global_oem
        df_filtrado = aplicar_filtros(
            rota_filter.value, status_filter.value, tipo_filter.value,
            executor_filter.value, origem_filter.value,
            tipocausa_filter.value,
            data_inicial_filter.value, data_final_filter.value
        )
        df_filtered_global_oem = df_filtrado

    aplicar_filter_button.on_click(on_button_click)

    display(data_inicial_filter)
    display(data_final_filter)
    df_filtered_global_oem
    display(
        widgets.HBox([origem_filter, tipo_filter, rota_filter, executor_filter, status_filter],
                     layout=widgets.Layout(align_items='baseline'))
    )
    display(
        widgets.HBox([tipocausa_filter, aplicar_filter_button],
                     layout=widgets.Layout(align_items='center'))
    )

DatePicker(value=datetime.date(2022, 1, 1), step=1)

DatePicker(value=datetime.date(2024, 5, 9), step=1)

In [6]:
df_filtered_global_oem.shape

(121, 52)

In [7]:
df_table =df_filtered_global_oem.rename(columns={'IDSIGFI':'CÓDIGO ONE','UC': 'NÚM.DA UC','OBRA':'NÚM. DA OBRA','NOMECLIENTE':'NOME DO CLIENTE','CPF':'CPF','ENDERECO':'ENDEREÇO',
                                                   'MUNICIPIO':'MUNICÍPIO','TIPOCAUSA':'CORRETIVA','EXECUTOR':'EQUIPE EXECUTORA','OBRA':'NÚM.DA OBRA','ROTA':'ROTA','TIPODESIGFI': 'TIPO/SIGFI','DTCONCLUSAO':'DATA ATENDIMENTO'})
                                                     

replacements1 = {'ONEOM001':'EQ.1','ONEOM002':'EQ.2','ONEOM003':'EQ.3','ONEOM004':'EQ.4','ONEOM005':'EQ.5','ONEOM006':'EQ.6','ONEOM007':'EQ.7',
                 'ONEOM008':'EQ.8','ONEOM009':'EQ.9','ONEOM010':'EQ.10','ONEOM011':'EQ.11','ONEOM012':'EQ.12','ONEOM013':'EQ.13','ONEOM014':'EQ.14',
                 'ONEOM015':'EQ.15'}
df_table['EQUIPE EXECUTORA'] = df_table['EQUIPE EXECUTORA'].replace(replacements1)

replacements2 = {'INTERMITENICA':'CORRETIVA'}
df_table['CORRETIVA'] = df_table['CORRETIVA'].replace(replacements2)

replacements3 = {1:'CRUZEIRO DO SUL', 2:'CRUZEIRO DO SUL', 3:'CRUZEIRO DO SUL',
                 4:'CRUZEIRO DO SUL', 5:'CRUZEIRO DO SUL', 6:'CRUZEIRO DO SUL',
                 7:'CRUZEIRO DO SUL', 8:'CRUZEIRO DO SUL', 9:'CRUZEIRO DO SUL',
                 10:'CRUZEIRO DO SUL',13:'CRUZEIRO DO SUL',12:'BRASILÉIA',
                 11:'RIO BRANCO', 14:'CRUZEIRO DO SUL', 15:'CRUZEIRO DO SUL', 
                 16:'CRUZEIRO DO SUL', 17:'CRUZEIRO DO SUL',21:'TARAUACÁ',
                 22:'CRUZEIRO DO SUL',23:'TARAUACÁ', 24:'CRUZEIRO DO SUL'}

df_table['REGIONAL'] = df_table['ROTA'].replace(replacements3)

df_table = df_table[['CÓDIGO ONE', 'NÚM.DA UC', 'NÚM.DA OBRA', 'NOME DO CLIENTE', 'CPF',
                       'ENDEREÇO', 'MUNICÍPIO','REGIONAL', 'DATA ATENDIMENTO', 'CORRETIVA', 'ROTA',
                       'EQUIPE EXECUTORA', 'TIPO/SIGFI', 'NUMOS']]

In [8]:
#excell = r"E:\OEM\content\base_sip_Concluido.xlsx"
#dataframe = pd.read_excel(excell)
df_movement = pd.read_excel(path_excel_mov)
df_componentlist = pd.read_excel(path_excel_comp)
df_date = df_filtered_global_oem[['NUMOS','NUMOCORRENCIA','EXECUTOR','STATUS', 'LATLONCON','DTAHORARECLAMACAO','DTCONCLUSAO','CONCLUSAO']]
table_dt = df_date.copy()
table_dt['DTAHORARECLAMACAO'] = pd.to_datetime(table_dt['DTAHORARECLAMACAO'],errors='coerce')
table_dt['DTCONCLUSAO'] = pd.to_datetime(table_dt['DTCONCLUSAO'])
table_dt['MES ABERTURA'] = table_dt['DTAHORARECLAMACAO'].dt.month
table_dt['ANO ABERTURA'] = table_dt['DTAHORARECLAMACAO'].dt.year
table_dt['MES CONCLUSAO'] = table_dt['DTCONCLUSAO'].dt.month
table_dt['ANO CONCLUSAO'] = table_dt['DTCONCLUSAO'].dt.year
table_dt['HORA CONCLUSAO'] = table_dt['DTCONCLUSAO'].dt.strftime('%H:%M')
#dataset[['LATITUDE', 'LONGITUDE','ERRO_1','ERRO_2']] = dataset['LATLONCON'].apply(lambda x: pd.Series(x.split(',')))#,'ERRO_1','ERRO_2'
def split_latlon(x):
    if isinstance(x, str):
        return pd.Series(x.split(','))
    else:
        #Se não for uma string, retorna NaN ou valores padrão, dependendo do que você deseja
        return pd.Series([pd.NaT, pd.NaT])

# Aplica a função split_latlon à coluna 'LATLONCON'
table_dt[['LATITUDE', 'LONGITUDE']] = table_dt['LATLONCON'].apply(split_latlon)#,'erro_1','erro_2'

replacements_1 = {'ONEOM001':'EQ.1','ONEOM002':'EQ.2','ONEOM003':'EQ.3','ONEOM004':'EQ.4','ONEOM005':'EQ.5','ONEOM006':'EQ.6','ONEOM007':'EQ.7',
                 'ONEOM008':'EQ.8','ONEOM009':'EQ.9','ONEOM010':'EQ.10','ONEOM011':'EQ.11','ONEOM012':'EQ.12','ONEOM013':'EQ.13','ONEOM014':'EQ.14',
                 'ONEOM015':'EQ.15'}
table_dt['EQUIPE'] = table_dt['EXECUTOR'].replace(replacements_1)

# Calculando as colunas data de fechamento e abertura
today = datetime.now()
#Aplica a formula para função do excell =SE(AD921="";HOJE()-AA921;AD921-AD921)
def days_of_late(row):
    if pd.isnull(row['DTCONCLUSAO']):
        return today - row['DTCONCLUSAO']
    else:
        return row['DTCONCLUSAO'] - row['DTAHORARECLAMACAO']
        days = diff.days
        hours = diff.seconds // 3600
        return f'{days} dias e {hours} horas'
#Aplicando a função
table_dt['CONCLUSAO - ABERTURA'] = table_dt.apply(days_of_late, axis=1)

In [9]:
table_dt

,NUMOS,NUMOCORRENCIA,EXECUTOR,STATUS,LATLONCON,DTAHORARECLAMACAO,DTCONCLUSAO,CONCLUSAO,MES ABERTURA,ANO ABERTURA,MES CONCLUSAO,ANO CONCLUSAO,HORA CONCLUSAO,LATITUDE,LONGITUDE,EQUIPE,CONCLUSAO - ABERTURA
52,202400029205,65920,ONEOM014,VALIDADO,"-8.53816,-70.91948000000001",2024-04-25 15:44:00,2024-04-26 15:53:00,NORMALIZADO SEM SUBST DE EQUIPAMENTO,4,2024,4,2024,15:53,-8.53816,-70.91948000000001,EQ.14,1 days 00:09:00
53,202400029197,65479,ONEOM014,VALIDADO,"-8.501666666666667,-70.85774166666667",2024-04-25 15:42:00,2024-04-27 08:28:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,4,2024,4,2024,08:28,-8.501666666666667,-70.85774166666667,EQ.14,1 days 16:46:00
94,202400028330,61557,ONEOM014,VALIDADO,"-8.46365,-70.82482833333333",2024-04-24 11:25:00,2024-04-27 10:55:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,4,2024,4,2024,10:55,-8.46365,-70.82482833333333,EQ.14,2 days 23:30:00
95,202400028322,61436,ONEOM006,VALIDADO,"-8.4645698,-70.81340851",2024-04-24 11:22:00,2024-04-27 09:24:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,4,2024,4,2024,09:24,-8.4645698,-70.81340851,EQ.6,2 days 22:02:00
96,202400028314,61482,ONEOM006,VALIDADO,"-8.50122579,-70.85882661",2024-04-24 11:18:00,2024-04-26 10:36:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,4,2024,4,2024,10:36,-8.50122579,-70.85882661,EQ.6,1 days 23:18:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2337,202400000487,NaN,ONEOM014,VALIDADO,"-8.49237,-70.836495",2024-01-04 14:37:00,2024-01-08 10:37:00,NORMALIZADO SEM SUBST DE EQUIPAMENTO,1,2024,1,2024,10:37,-8.49237,-70.836495,EQ.14,3 days 20:00:00
2355,202400000305,NaN,ONEOM014,VALIDADO,"-8.71459,-71.01432666666666",2024-01-03 10:00:00,2024-01-05 15:47:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,1,2024,1,2024,15:47,-8.71459,-71.01432666666666,EQ.14,2 days 05:47:00
2356,202400000297,NaN,ONEOM014,VALIDADO,"-8.415608333333333,-70.82019833333334",2024-01-03 09:58:00,2024-01-04 11:53:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,1,2024,1,2024,11:53,-8.415608333333333,-70.82019833333334,EQ.14,1 days 01:55:00
2357,202400000289,171184,ONEOM014,VALIDADO,"-8.698325,-71.01141333333334",2024-01-03 09:54:00,2024-01-05 13:12:00,NORMALIZADO COM SUBST DE EQUIPAMENTO,1,2024,1,2024,13:12,-8.698325,-71.01141333333334,EQ.14,2 days 03:18:00


In [10]:
component_install = df_movement.loc[(df_movement['TIPO1'] =='INSTALADO')]
# Criando uma função para realizar a contagem de componentes
def count_component(component_install):
    descrition = []

    for idservico in component_install['NUMOS'].unique():
        subset_component = component_install[component_install['NUMOS'] == idservico]        
        count_component = subset_component['DESCRICAO'].value_counts()
        format_descrition = ', '.join(f'Troca de {count} {component}' if count > 1 else f'Troca de 1 {component}' for component, count in count_component.items())
        descrition.append(format_descrition)

    df_descrition= pd.DataFrame({'NUMOS':component_install['NUMOS'].unique(), 'DESCRICAO_FINAL':descrition})
    return df_descrition
observation =  count_component(component_install)
table_dt = table_dt.merge(observation, left_on='NUMOS', right_on='NUMOS', how='left')

In [11]:
table_two = numocorrencia.merge(table_dt, on='NUMOCORRENCIA', how='left')
table_one = table_two.merge(df_table, on='NUMOS', how='left')

In [12]:
plan_one = table_one[['CÓDIGO ONE', 'NÚM.DA UC', 'NÚM.DA OBRA', 'NOME DO CLIENTE',
                       'CPF', 'ENDEREÇO', 'MUNICÍPIO', 'REGIONAL', 'DATA ATENDIMENTO',
                        'CORRETIVA', 'ROTA', 'EQUIPE EXECUTORA', 'TIPO/SIGFI', 'LATITUDE', 
                        'LONGITUDE','NUMOS']]
plan_two = table_two[['NUMOCORRENCIA', 'DTAHORARECLAMACAO','MES ABERTURA','ANO ABERTURA','DTCONCLUSAO','CONCLUSAO - ABERTURA', 
                      'MES CONCLUSAO','ANO CONCLUSAO', 'HORA CONCLUSAO','EQUIPE','STATUS', 'LATITUDE', 
                      'LONGITUDE', 'CONCLUSAO','DESCRICAO_FINAL']]
plan_two['DESCRICAO_FINAL'].replace('NORMALIZADO SEM SUBST DE EQUIPAMENTO', 'NORMALIZADO COM SUBST DE EQUIPAMENTO')
plan_two.loc[plan_two['CONCLUSAO'] == 'NORMALIZADO SEM SUBST DE EQUIPAMENTO', 'DESCRICAO_FINAL'] = 'Não houve troca de componentes'

In [13]:
plan_one

,CÓDIGO ONE,NÚM.DA UC,NÚM.DA OBRA,NOME DO CLIENTE,CPF,ENDEREÇO,MUNICÍPIO,REGIONAL,DATA ATENDIMENTO,CORRETIVA,ROTA,EQUIPE EXECUTORA,TIPO/SIGFI,LATITUDE,LONGITUDE,NUMOS
0,16759,713716.0,212300948.0,MARIA ROSICLEIDE XAVIER DOS SANTOS,65413970234,COMUNIDADE TARTARUGA S/N IGARAPE VALPARAÍSO C...,CRUZEIRO DO SUL,CRUZEIRO DO SUL,2024-04-17 15:51:00,INTERMITENCIA,16.0,EQ.4,SIGFI80,-8.09141552,-72.60602531,202400026276
1,16742,713949.0,212300942.0,MARIA DAS GRACAS OLIVEIRA LIMA,62871269220,COMUNIDADE TERRA FIRME 1 S/N IGARAPE VALPARAÍS...,CRUZEIRO DO SUL,CRUZEIRO DO SUL,2024-04-17 09:27:00,INTERMITENCIA,16.0,EQ.4,SIGFI80,-8.00125681,-72.72243252,202400026250
2,18707,713951.0,212301994.0,MARCELO MELO DE CARVALHO,71857150244,COMUNIDADE FOZ DO VALPARAÍSO COMUNIDADE FOZ...,CRUZEIRO DO SUL,CRUZEIRO DO SUL,2024-04-27 12:55:00,INTERMITENCIA,16.0,EQ.4,SIGFI80,-8.025763333333334,-72.745425,202400026391
3,16693,714062.0,212300955.0,ANDREILSON BARBOSA MEIRELES,9758541277,SANTA LUZIA S/N IGARAPE VALPARAÍSO SANTA LUZIA,CRUZEIRO DO SUL,CRUZEIRO DO SUL,2024-04-18 09:45:00,INTERMITENCIA,16.0,EQ.4,SIGFI80,-8.15845374,-72.57229605,202400026201
4,16745,714177.0,212300982.0,MARIA ELIANE SILVA DE OLIVEIRA,81087985234,COMUNIDADE TERRA FIRME S/N IGARAPE VALPARAÍSO ...,CRUZEIRO DO SUL,CRUZEIRO DO SUL,2024-04-27 15:57:00,INTERMITENCIA,16.0,EQ.4,SIGFI80,-8.106661666666668,-72.59654333333333,202400026268
5,17362,714453.0,212301073.0,ELIDO SILVA FERNANDES,177667290,RIO AZUL COM BELA VISTA RIO AZUL COM BELA V...,NaN,CRUZEIRO DO SUL,2024-03-19 09:55:00,INTERMITENCIA,14.0,EQ.6,SIGFI80,-7.47760436,-73.27967318,202400011492
6,19727,714931.0,212301033.0,ELISEU DO NASCIMENTO FERREIRA,015103422-22,RIO MURU; SERINGAL OURO PRETO RIO MURU; SER...,TARAUACA,TARAUACÁ,2024-02-21 15:05:00,INTERMITENCIA,21.0,EQ.7,SIGFI80,-8.6151268,-70.94873227,202400009223
7,19007,715169.0,212301138.0,GENILCIA PINHEIRO OLIVEIRA,3949553231,PDS SÃO SALVADOR; COMUNIDADE PROSPERIDADE P...,MANCIO LIMA,CRUZEIRO DO SUL,2024-03-19 15:04:00,INTERMITENCIA,14.0,EQ.6,SIGFI80,-7.41989055,-73.22077636,202400014488
8,19657,715270.0,212301152.0,ALBANIR ALVES DE SOUZA,694685592-20,RIO MURÚ SERINGAL OURO PRETO RIO MURÚ SERIN...,TARAUACA,TARAUACÁ,2024-02-21 09:35:00,INTERMITENCIA,21.0,EQ.7,SIGFI80,-8.58290923,-70.92453715,202400009207
9,19751,715278.0,212301183.0,DIENE SOUZA ALVES,700952222-78,RIO MURÚ SERINGAL OURO PRETO RIO MURÚ SERIN...,TARAUACA,TARAUACÁ,2024-02-21 11:35:00,INTERMITENCIA,21.0,EQ.7,SIGFI80,-8.60032861,-70.92890862,202400011708


In [14]:
# Criar uma planilha base
wb = Workbook()
ws = wb.create_sheet('producao_corretiva')

# Escrever os dados do plan_one
row_offset = 1
for col_idx, col in enumerate(plan_one.columns, start=1):
    ws.cell(row=row_offset, column=col_idx, value=col)

for r_idx, row in plan_one.iterrows():
    for col_idx, value in enumerate(row, start=1):
        ws.cell(row=row_offset + r_idx + 1, column=col_idx, value=value)

# Atualizar o deslocamento para a próxima coluna
column_offset = plan_one.shape[1] + 2

# Escrever os dados do plan_two ao lado do plan_one
for col_idx, col in enumerate(plan_two.columns, start=column_offset):
    ws.cell(row=1, column=col_idx, value=col)

for r_idx, row in plan_two.iterrows():
    for col_idx, value in enumerate(row, start=column_offset):
        ws.cell(row=r_idx + 2, column=col_idx, value=value)

# Salvar a planilha
wb.save(r"C:\Users\helym\projetos_python\jupyter\OEM\output\2ºtrache_corretiva_lot01_1ªremessa_intelbras.xlsx")